In [1]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import xgboost as xgb
import sys
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [29]:
sys.path.append(os.path.abspath('../'))

# Agora você pode importar a função desejada
from Utils.Uteis import Modelo

In [30]:
app = Flask(__name__)

utilModelo = Modelo()

model = utilModelo.carregarModelo("../Modelo/modelo.joblib")

Modelo carregado de: ../Modelo/modelo.joblib


In [51]:
app = Flask(__name__)

# Função para preparar os dados, garantindo que todas as colunas estejam
def prepare_input(form_data):
    expected_columns = [
        "Current Loan Amount", "Credit Score", "Annual Income", "Years in current job",
        "Monthly Debt", "Years of Credit History", "Number of Open Accounts",
        "Number of Credit Problems", "Current Credit Balance", "Maximum Open Credit",
        "Bankruptcies", "Tax Liens", "TermLabel",
        "Home Ownership_HaveMortgage", "Home Ownership_Home Mortgage",
        "Home Ownership_Own Home", "Home Ownership_Rent",
        "Purpose_Business Loan", "Purpose_Buy House", "Purpose_Buy a Car",
        "Purpose_Debt Consolidation", "Purpose_Educational Expenses",
        "Purpose_Home Improvements", "Purpose_Medical Bills", "Purpose_Other",
        "Purpose_Take a Trip", "Purpose_major_purchase", "Purpose_moving",
        "Purpose_other", "Purpose_renewable_energy", "Purpose_small_business",
        "Purpose_vacation", "Purpose_wedding"
    ]

    # Inicializa zeros
    input_data = {col: 0 for col in expected_columns}

    # Mapeia dados numéricos do form
    input_data["Current Loan Amount"] = float(form_data.get("loan_amount", 0))
    input_data["Credit Score"] = float(form_data.get("credit_score", 0))
    input_data["Annual Income"] = float(form_data.get("annual_income", 0))
    input_data["Years in current job"] = float(form_data.get("years_job", 0))
    input_data["Monthly Debt"] = float(form_data.get("monthly_debt", 0))
    input_data["Years of Credit History"] = float(form_data.get("credit_history", 0))
    input_data["Number of Open Accounts"] = float(form_data.get("open_accounts", 0))
    input_data["Number of Credit Problems"] = float(form_data.get("credit_problems", 0))
    input_data["Current Credit Balance"] = float(form_data.get("credit_balance", 0))
    input_data["Maximum Open Credit"] = float(form_data.get("max_credit", 0))
    input_data["Bankruptcies"] = float(form_data.get("bankruptcies", 0))
    input_data["Tax Liens"] = float(form_data.get("tax_liens", 0))

    # TermLabel: 1 para Short Term, 0 para Long Term
    input_data["TermLabel"] = 1 if form_data.get("loan_term") == "Short Term" else 0

    # Home Ownership (one-hot)
    home_ownership = form_data.get("home_ownership", "")
    home_col = f"Home Ownership_{home_ownership}"
    if home_col in input_data:
        input_data[home_col] = 1

    # Purpose (one-hot)
    purpose = form_data.get("loan_purpose", "")
    purpose_col = f"Purpose_{purpose}"
    if purpose_col in input_data:
        input_data[purpose_col] = 1

    # Transforma em DataFrame
    input_df = pd.DataFrame([input_data])
    return input_df

@app.route('/')
def home():
    home_options = ['Own Home', 'Rent', 'HaveMortgage', 'Home Mortgage']
    purpose_options = [
        'Business Loan', 'Buy House', 'Buy a Car', 'Debt Consolidation', 
        'Educational Expenses', 'Home Improvements', 'Medical Bills',
        'Other', 'Take a Trip', 'major_purchase', 'moving', 'other',
        'renewable_energy', 'small_business', 'vacation', 'wedding'
    ]
    return render_template('index.html', home_options=home_options, purpose_options=purpose_options)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        form_data = request.form.to_dict()
        input_df = prepare_input(form_data)

        prediction = model.predict(input_df)
        
        # Se seu modelo suporta probabilidade (ex: model.predict_proba)
        if hasattr(model, 'predict_proba'):
            proba = model.predict_proba(input_df)[0][1]  # probabilidade classe 1
            probability = f"{proba:.2f}"
        else:
            probability = "N/A"

        result = "Negado" if prediction[0] == 1 else "Aprovado"

        # converter input_df para dict simples para enviar ao template
        features = input_df.iloc[0].to_dict()

        return render_template("result.html", result=result, probability=probability, features=features)
    except Exception as e:
        return f"Erro: {str(e)}"



In [52]:
if __name__ == '__main__':
    app.run(use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jun/2025 11:27:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 11:27:53] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [06/Jun/2025 11:28:36] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [06/Jun/2025 11:28:36] "GET /static/css/style.css HTTP/1.1" 304 -
